<a href="https://colab.research.google.com/github/popudrak/DSC-PJATK/blob/main/implicit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 45.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import bm25_weight
import json
from implicit.als import AlternatingLeastSquares

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

with open("id_mappings.json", "r") as f:
    id_mappings = json.load(f)

item_mapping_json = id_mappings["item_mapping"]
item_reverse_mapping_json = id_mappings["item_reverse_mapping"]
user_mapping_json = id_mappings["user_mapping"]
user_reverse_mapping_json = id_mappings["user_reverse_mapping"]

all_product_ids = [str(k) for k in item_mapping_json.keys()]
all_user_ids = [f"user_{str(k)}" for k in user_mapping_json.keys()]

item_mapping_manual = {pid: idx for idx, pid in enumerate(all_product_ids)}
reverse_item_mapping_manual = {idx: pid for pid, idx in item_mapping_manual.items()}
user_mapping_manual = {uid: idx for idx, uid in enumerate(all_user_ids)}
reverse_user_mapping_manual = {idx: uid for uid, idx in user_mapping_manual.items()}

print(f"Liczba produktów z mappingu JSON: {len(all_product_ids)}")
print(f"Liczba użytkowników z mappingu JSON: {len(all_user_ids)}")

assert len(all_product_ids) == len(item_mapping_json), "Niespójność w mapowaniu produktów!"
assert len(all_user_ids) == len(user_mapping_json), "Niespójność w mapowaniu użytkowników!"

Liczba produktów z mappingu JSON: 77833
Liczba użytkowników z mappingu JSON: 868218


In [ ]:
train['product_id_str'] = train['item_id'].map(lambda x: str(item_reverse_mapping_json[str(x)]))
train['user_id_str'] = train['user_id'].map(lambda x: f"user_{str(user_reverse_mapping_json[str(x)])}")

train['item_idx'] = train['product_id_str'].map(item_mapping_manual)
train['user_idx'] = train['user_id_str'].map(user_mapping_manual)

assert train['item_idx'].isna().sum() == 0, "Są produkty w danych treningowych, których nie ma w mappingu!"
assert train['user_idx'].isna().sum() == 0, "Są użytkownicy w danych treningowych, których nie ma w mappingu!"

print("Mapowanie danych treningowych poprawne.")

Mapowanie danych treningowych poprawne.


In [ ]:
interactions = sp.coo_matrix(
    (train['rating'].astype(np.float32), (train['item_idx'], train['user_idx'])),
    shape=(len(all_product_ids), len(all_user_ids))
).tocsr()

interactions_weighted = bm25_weight(interactions, K1=100, B=0.9)
user_item_matrix = interactions_weighted.T.tocsr()

print(f"Rozmiar macierzy interakcji: {interactions.shape}")
assert interactions.shape[0] == len(all_product_ids), "Nieprawidłowy rozmiar macierzy - produkty!"
assert interactions.shape[1] == len(all_user_ids), "Nieprawidłowy rozmiar macierzy - użytkownicy!"

Rozmiar macierzy interakcji: (77833, 868218)


In [ ]:
print("Rozmiar user-item matrix:", user_item_matrix.shape)
assert user_item_matrix.shape[1] == len(all_product_ids), "Liczba produktów nie zgadza się!"
assert user_item_matrix.shape[0] == len(all_user_ids), "Liczba użytkowników nie zgadza się!"

Rozmiar user-item matrix: (868218, 77833)


In [ ]:
model = AlternatingLeastSquares(factors=128, regularization=0.1, iterations=50)
model.fit(user_item_matrix)

print(f"Model factors shape (produkty x wymiary): {model.item_factors.shape}")
assert model.item_factors.shape[0] == len(all_product_ids), "Model wytrenowany na niewłaściwej liczbie produktów!"

  0%|          | 0/50 [00:00<?, ?it/s]

Model factors shape (produkty x wymiary): (77833, 128)


In [ ]:
all_predicted_products = []

for pred in submission_df['predictions']:
    all_predicted_products.extend(pred.split(" "))

all_predicted_products = set(all_predicted_products)
print(f"Liczba unikalnych produktów w predykcjach: {len(all_predicted_products)}")

# Zakres produktów znanych z mappingu
known_product_ids = set(item_reverse_mapping_json.values())
print(f"Liczba znanych produktów w mappingu: {len(known_product_ids)}")

# Czy wszystkie produkty z predykcji są faktycznie znane?
unknown_products = [p for p in all_predicted_products if p not in known_product_ids]
print(f"Liczba nieznanych produktów w predykcjach: {len(unknown_products)}")

Liczba unikalnych produktów w predykcjach: 8332
Liczba znanych produktów w mappingu: 77833
Liczba nieznanych produktów w predykcjach: 0


In [ ]:
submission = []
user_seen_items = train.groupby('user_idx')['item_idx'].apply(set).to_dict()

test['user_id_str'] = test['user_id'].map(lambda x: f"user_{str(user_reverse_mapping_json[str(x)])}")
test_user_indices = test['user_id_str'].map(user_mapping_manual)

missing_users = test_user_indices.isna().sum()
print(f"Liczba nieznanych użytkowników w test.csv: {missing_users}")

for uid, uidx in zip(test['user_id'], test_user_indices):
    if pd.isna(uidx):
        top_items = list(item_mapping_manual.values())[:10]
    else:
        seen = user_seen_items.get(uidx, set())

        recommended = model.recommend(
            int(uidx),
            user_item_matrix[int(uidx)],
            N=20,
            filter_items=list(seen) if seen else None
        )

        top_idxs = []
        for idx in recommended[:10]:
            if isinstance(idx, (np.ndarray, list)):
                top_idxs.append(int(idx[0]))
            else:
                top_idxs.append(int(idx))

        assert all(i in reverse_item_mapping_manual for i in top_idxs), "Model zwrócił nieistniejący indeks produktu!"
        top_items = [reverse_item_mapping_manual[i] for i in top_idxs]

    top_items_clean = [pid.replace("prod_", "") for pid in top_items]
    submission.append((uid, " ".join(top_items_clean)))

submission_df = pd.DataFrame(submission, columns=["user_id", "predictions"])
submission_df.to_csv("submission_implicit_bpr_bm25.csv", index=False)
print("Plik submission wygenerowany.")

Liczba nieznanych użytkowników w test.csv: 0
Plik submission wygenerowany.


In [ ]:
submission_df['predictions'].nunique()

19681

In [ ]:
product_popularity = train['item_idx'].value_counts().reset_index()
product_popularity.columns = ['item_idx', 'interactions']

print(product_popularity.head(20))

    item_idx  interactions
0      20832         28303
1      12595         22787
2      12372         16378
3       6864         15428
4      33154         14792
5      13451         13007
6       7030         12306
7      24931         12188
8      18550         11628
9      12104         11551
10     27505         11327
11     10719         11140
12     50073         10777
13     32523         10545
14     28271         10319
15     12895         10089
16     65542          9816
17     21770          9732
18     41330          9584
19     44247          9360
